# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [102]:
import pandas as pd
import os
import numpy as np

In [103]:
DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
**Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.**

### Strategy

We will import and wrangle the data for each country separately and then combine everything into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

In [104]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

In [105]:
def associate_to_country(country, df):
    """Adds a country super index to the provided dataframe"""
    country = pd.DataFrame({
        'Country': np.repeat(country, len(df)),
        'Date': df.index
    })
    return country.merge(df, right_index=True, left_on='Date').set_index(['Country', 'Date'])

### Guinea

In [106]:
def parse_num(x):
    try:
        return np.int32(x.strip())
    except ValueError:
        print('Unreadable number', x)
        return np.nan

In [107]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'New deaths registered today (confirmed)': 'Death confirmed',
    'New deaths registered today (probables)': 'Death probable',
    'New deaths registered today (suspects)': 'Death suspected',
    'New cases of confirmed': 'New cases confirmed',
    'New cases of probables': 'New cases probable', 
    'New cases of suspects': 'New cases suspected'
}
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Description', 'Totals'],
                     converters={'Totals': parse_num})
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    # Check for duplicates
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

Unreadable number 63%
Unreadable number 63%
Unreadable number 63%
Unreadable number 60%
Unreadable number 65%
Unreadable number 62%
Unreadable number 63%
Unreadable number 61%
Unreadable number 62%
Unreadable number 65%
Unreadable number 64%
Unreadable number 65%
Unreadable number 63%
Unreadable number 65%
Unreadable number 66%
Unreadable number 
Unreadable number 61%
Unreadable number 61%
Unreadable number 
Unreadable number 66%
Unreadable number 61%
Unreadable number 65%
Unreadable number 66%


In [108]:
tables_with_duplicates

[]

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [109]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [110]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date
guinea = associate_to_country('Guinea', guinea).rename(columns=interesting_fields)

In [111]:
guinea.head(n=10)

New cases confirmed  New cases probable  \
Country Date                                                  
Guinea  2014-08-04                  4.0                 0.0   
        2014-08-26                 10.0                 NaN   
        2014-08-27                 10.0                 0.0   
        2014-08-30                  9.0                 0.0   
        2014-08-31                 29.0                 8.0   
        2014-09-02                 14.0                 0.0   
        2014-09-04                 17.0                 0.0   
        2014-09-07                 11.0                 0.0   
        2014-09-08                 11.0                 0.0   
        2014-09-09                  7.0                 0.0   

                    New cases suspected  Death confirmed  Death probable  \
Country Date                                                               
Guinea  2014-08-04                  5.0              2.0             0.0   
        2014-08-26                 18.0              NaN             NaN   
        2014-08-27                 12.0              NaN             NaN   
        2014-08-30                 15.0              NaN             NaN   
        2014-08-31                  9.0              NaN             NaN   
        2014-09-02                 11.0              NaN             NaN   
        2014-09-04                 13.0              NaN             NaN   
        2014-09-07                  5.0              NaN             NaN   
        2014-09-08                  5.0              NaN             NaN   
        2014-09-09                  9.0              NaN             NaN   

                    Death suspected  
Country Date                         
Guinea  2014-08-04              0.0  
        2014-08-26              NaN  
        2014-08-27              NaN  
        2014-08-30              NaN  
        2014-08-31              NaN  
        2014-09-02              NaN  
        2014-09-04              NaN  
        2014-09-07              NaN  
        2014-09-08              NaN  
        2014-09-09              NaN

### Liberia

In [112]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'Total death/s in confirmed cases': 'Death confirmed',
    'Total death/s in probable cases': 'Death probable',
    'Total death/s in suspected cases': 'Death suspected',
    'New case/s (confirmed)': 'New cases confirmed',
    'New Case/s (Probable)': 'New cases probable',
    'New Case/s (Suspected)': 'New cases suspected'
}
for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Variable', 'National'],
                     converters={'National': parse_num})
    df = df[df.Variable.isin(interesting_fields)]
    # Check for duplicates
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable n

Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 66.7
Unreadab

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [113]:
tables_with_duplicates

['2014-10-04-v142.csv']

In [114]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
17,2014-10-04,Total death/s in confirmed cases,1012.0
18,2014-10-04,Total death/s in probable cases,699.0
19,2014-10-04,Total death/s in suspected cases,488.0
34,2014-10-04,Total death/s in confirmed cases,1018.0
35,2014-10-04,Total death/s in probable cases,701.0
36,2014-10-04,Total death/s in suspected cases,491.0


7 Variables are repeated twice.

In [115]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [116]:
second - first

,National
Variable,
Total death/s in confirmed cases,6.0
Total death/s in probable cases,2.0
Total death/s in suspected cases,3.0


We decided to keep the bigger values as the difference is not very relevant:

In [117]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [118]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [119]:
liberia = pd.concat(dfs).sort_index()
liberia = associate_to_country('Liberia', liberia).rename(columns=interesting_fields)

In [120]:
liberia.head(n=10)

New cases probable  New cases suspected  \
Country Date                                                  
Liberia 2014-06-16                 1.0                  2.0   
        2014-06-17                 0.0                  2.0   
        2014-06-22                 0.0                  5.0   
        2014-06-24                 0.0                  2.0   
        2014-06-25                 1.0                  4.0   
        2014-06-28                 6.0                  2.0   
        2014-06-29                 0.0                  0.0   
        2014-07-01                 0.0                  2.0   
        2014-07-02                 3.0                  1.0   
        2014-07-03                 2.0                  1.0   

                    New cases confirmed  Death confirmed  Death probable  \
Country Date                                                               
Liberia 2014-06-16                  1.0              8.0             6.0   
        2014-06-17                  0.0              8.0             6.0   
        2014-06-22                  5.0             16.0             8.0   
        2014-06-24                  4.0             18.0             8.0   
        2014-06-25                  2.0             20.0             8.0   
        2014-06-28                  1.0             26.0            10.0   
        2014-06-29                  2.0             26.0            10.0   
        2014-07-01                  2.0             30.0            17.0   
        2014-07-02                  0.0             32.0            20.0   
        2014-07-03                  1.0             33.0            22.0   

                    Death suspected  
Country Date                         
Liberia 2014-06-16              2.0  
        2014-06-17              2.0  
        2014-06-22              1.0  
        2014-06-24              6.0  
        2014-06-25              9.0  
        2014-06-28             13.0  
        2014-06-29             13.0  
        2014-07-01             14.0  
        2014-07-02             14.0  
        2014-07-03             15.0

### Sierra Leone

In [121]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'death_confirmed': 'Death confirmed',
    'death_probable': 'Death probable',
    'death_suspected': 'Death suspected',
    'new_confirmed': 'New cases confirmed',
    'new_probable': 'New cases probable',
    'new_suspected': 'New cases suspected'
}

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path,
                     parse_dates=['date'],
                     usecols=['date', 'variable', 'National'], 
                     converters={'National': parse_num})
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 33.2
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 98%
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 98%
Unreadable number 37.8
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 98%
Unreadable number 24.2
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable number 
Unreadable

In [122]:
tables_with_duplicates

[]

In [123]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [124]:
dfs[0]

Variable,death_confirmed,death_probable,death_suspected,new_confirmed,new_probable,new_suspected
Date,,,,,,
2014-10-25,1037.0,211.0,135.0,70.0,9.0,8.0


In [125]:
sl = pd.concat(dfs).sort_index()
sl = associate_to_country('Sierra Leone', sl).rename(columns=interesting_fields)

In [126]:
sl.head(n=10)

Death confirmed  Death probable  Death suspected  \
Country      Date                                                           
Sierra Leone 2014-08-12            264.0            34.0              5.0   
             2014-08-13            273.0            34.0              5.0   
             2014-08-14            280.0            34.0              5.0   
             2014-08-15            287.0            34.0              5.0   
             2014-08-16            297.0            34.0              5.0   
             2014-08-17            305.0            34.0              5.0   
             2014-08-18            312.0            34.0              5.0   
             2014-08-19            320.0            34.0              5.0   
             2014-08-20            322.0            34.0              5.0   
             2014-08-21            329.0            34.0              8.0   

                         New cases confirmed  New cases probable  \
Country      Date                                                  
Sierra Leone 2014-08-12                 11.0                 1.0   
             2014-08-13                 15.0                 1.0   
             2014-08-14                 13.0                 2.0   
             2014-08-15                 10.0                 1.0   
             2014-08-16                 18.0                 0.0   
             2014-08-17                  2.0                 0.0   
             2014-08-18                  5.0                15.0   
             2014-08-19                  9.0                 3.0   
             2014-08-20                  4.0                 0.0   
             2014-08-21                  9.0                 0.0   

                         New cases suspected  
Country      Date                             
Sierra Leone 2014-08-12                 10.0  
             2014-08-13                  3.0  
             2014-08-14                  0.0  
             2014-08-15                  6.0  
             2014-08-16                  3.0  
             2014-08-17                  1.0  
             2014-08-18                 40.0  
             2014-08-19                 16.0  
             2014-08-20                  1.0  
             2014-08-21                  0.0

In [127]:
del dfs
del tables_with_duplicates

### Combining the data

In [128]:
# Concatenate all dataframes
df = pd.concat([guinea, liberia, sl])

In [129]:
df.head(n=20)

Death confirmed  Death probable  Death suspected  \
Country Date                                                           
Guinea  2014-08-04              2.0             0.0              0.0   
        2014-08-26              NaN             NaN              NaN   
        2014-08-27              NaN             NaN              NaN   
        2014-08-30              NaN             NaN              NaN   
        2014-08-31              NaN             NaN              NaN   
        2014-09-02              NaN             NaN              NaN   
        2014-09-04              NaN             NaN              NaN   
        2014-09-07              NaN             NaN              NaN   
        2014-09-08              NaN             NaN              NaN   
        2014-09-09              NaN             NaN              NaN   
        2014-09-11              NaN             NaN              NaN   
        2014-09-14              NaN             NaN              NaN   
        2014-09-16              NaN             NaN              NaN   
        2014-09-17              NaN             NaN              NaN   
        2014-09-19              NaN             NaN              NaN   
        2014-09-21              NaN             NaN              NaN   
        2014-09-22              NaN             NaN              NaN   
        2014-09-23              NaN             NaN              NaN   
        2014-09-24              NaN             NaN              NaN   
        2014-09-26              NaN             NaN              NaN   

                    New cases confirmed  New cases probable  \
Country Date                                                  
Guinea  2014-08-04                  4.0                 0.0   
        2014-08-26                 10.0                 NaN   
        2014-08-27                 10.0                 0.0   
        2014-08-30                  9.0                 0.0   
        2014-08-31                 29.0                 8.0   
        2014-09-02                 14.0                 0.0   
        2014-09-04                 17.0                 0.0   
        2014-09-07                 11.0                 0.0   
        2014-09-08                 11.0                 0.0   
        2014-09-09                  7.0                 0.0   
        2014-09-11                 13.0                 0.0   
        2014-09-14                 14.0                11.0   
        2014-09-16                  6.0                 0.0   
        2014-09-17                  6.0                 0.0   
        2014-09-19                 12.0                 1.0   
        2014-09-21                 17.0                 0.0   
        2014-09-22                 14.0                 0.0   
        2014-09-23                 25.0                 0.0   
        2014-09-24                 20.0                 0.0   
        2014-09-26                 10.0                 7.0   

                    New cases suspected  
Country Date                             
Guinea  2014-08-04                  5.0  
        2014-08-26                 18.0  
        2014-08-27                 12.0  
        2014-08-30                 15.0  
        2014-08-31                  9.0  
        2014-09-02                 11.0  
        2014-09-04                 13.0  
        2014-09-07                  5.0  
        2014-09-08                  5.0  
        2014-09-09                  9.0  
        2014-09-11                  9.0  
        2014-09-14                  0.0  
        2014-09-16                  4.0  
        2014-09-17                  4.0  
        2014-09-19                  3.0  
        2014-09-21                  1.0  
        2014-09-22                  5.0  
        2014-09-23                  4.0  
        2014-09-24                  8.0  
        2014-09-26                  2.0

### Computing means

In [131]:
import calendar

# Compute mean per month per country. 
results = df.fillna(value=0)\ # WARNING: TO CHECK
            .groupby(lambda row: (row[0], row[1].month))\
            .mean()

# Give computation result a nice index
results.index = pd.MultiIndex.from_tuples(results.index, names=['Country', 'Month'])\
                  .map(lambda i: (i[0], calendar.month_name[i[1]]))

In [134]:
results

Death confirmed  Death probable  Death suspected  \
Guinea       August            0.400000        0.000000         0.000000   
             September         0.000000        0.000000         0.000000   
             October           0.000000        0.000000         0.000000   
Liberia      June             17.428571        8.000000         6.571429   
             July             43.272727       33.545455        17.636364   
             August          195.777778      215.000000       107.333333   
             September       606.750000      494.416667       316.125000   
             October         247.760000      140.040000        97.000000   
             November          0.000000        0.000000         0.000000   
             December          0.000000        0.000000         0.000000   
Sierra Leone August          293.850000       31.650000         6.450000   
             September       443.310345       34.448276        10.241379   
             October         905.464286      159.500000        97.392857   
             November       1122.904762      188.190476       142.380952   
             December       1314.800000      166.400000       126.400000   

                        New cases confirmed  New cases probable  \
Guinea       August               12.400000            1.600000   
             September            13.000000            1.187500   
             October               6.000000            0.000000   
Liberia      June                  2.142857            1.142857   
             July                  1.818182            3.727273   
             August                5.444444           19.777778   
             September             6.166667           29.333333   
             October               1.360000           17.760000   
             November              2.600000            7.000000   
             December           1928.333333         1208.000000   
Sierra Leone August               18.550000            1.150000   
             September            34.413793            0.000000   
             October              56.714286            0.857143   
             November             59.904762            0.000000   
             December             32.600000            0.000000   

                        New cases suspected  
Guinea       August               11.800000  
             September             5.437500  
             October              28.000000  
Liberia      June                  2.428571  
             July                  3.000000  
             August               12.000000  
             September            28.333333  
             October              26.440000  
             November             16.866667  
             December           2042.222222  
Sierra Leone August                5.450000  
             September             6.275862  
             October              13.357143  
             November             15.333333  
             December              8.400000

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here